In [1]:
import paddle.fluid as fluid
import paddle
import numpy as np
import paddle.dataset.uci_housing as uci_housing
paddle.enable_static()

In [2]:
x=fluid.data(name='x',shape=[None,13],dtype='float32')
hidden=fluid.layers.fc(input=x,size=100,act='relu')
hidden=fluid.layers.fc(input=hidden,size=100,act='relu')
net=fluid.layers.fc(input=hidden,size=1,act=None)
infer_program=fluid.default_main_program().clone(for_test=True)

In [3]:
y=fluid.data(name='y',shape=[None,1],dtype='float32')
cost=fluid.layers.square_error_cost(input=net,label=y)
avg_cost=fluid.layers.mean(cost)
optimizer=fluid.optimizer.SGDOptimizer(learning_rate=0.01)
opts=optimizer.minimize(avg_cost)

In [4]:
place=fluid.CUDAPlace(0)
exe=fluid.Executor(place)
exe.run(fluid.default_startup_program())

In [5]:
train_reader=paddle.batch(reader=uci_housing.train(),batch_size=128)
test_reader=paddle.batch(reader=uci_housing.test(),batch_size=128)
feeder=fluid.DataFeeder(place=place,feed_list=[x,y])

这里直接调用接口获取数据，在实际操作中，我们用列表保存每个batch_size的数据，并用字典进行feed

In [ ]:
for pass_id in range(300):
    train_cost=0
    for batch_id,data in enumerate(train_reader()):
        train_cost=exe.run(program=fluid.default_main_program(),
            feed=feeder.feed(data),
            fetch_list=[avg_cost])
    print("Pass:%d, Cost:%0.5f" %(pass_id, train_cost[0][-1]))

In [10]:
for data in test_reader():
    infer_data=np.array([data[0] for data in data]).astype("float32")
    infer_label=np.array([data[1] for data in data]).astype("float32")
    infer_result=exe.run(program=infer_program,
            feed={x.name:infer_data},fetch_list=[net])
    for i in range(len(infer_label)):
        print("预测结果 ：%f" %(infer_result[0][i]))